In [2]:
!pip install pathway bokeh --quiet

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bokeh')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [5]:
df = pd.read_csv('/content/dataset.csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [6]:
df.isna().sum()

,0
ID,0
SystemCodeNumber,0
Capacity,0
Latitude,0
Longitude,0
Occupancy,0
VehicleType,0
TrafficConditionNearby,0
QueueLength,0
IsSpecialDay,0


In [7]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

In [9]:
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,OccupancyRate
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.105719
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.197500
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.384279
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.513402
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.319792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.793617
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,1.000000
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.334489
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.538511


In [10]:
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]].to_csv("parking_stream.csv", index=False)

In [27]:
import pathway as pw

In [28]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    LastUpdatedDate: str
    LastUpdatedTime: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: str
    VehicleType: str

In [31]:
input_table = pw.io.csv.read(
    "dataset.csv",
    schema=ParkingSchema,
    mode="static",
)

In [33]:
traffic_map = {"Low": 1, "Medium": 2, "High": 3}
vehicle_weights = {"car": 0.1, "bike": 0.05, "truck": 0.2}
BASE_PRICE = 10.0
α, β, γ, δ, λ = 0.5, 0.3, 0.2, 0.4, 0.6

cleaned = input_table.select(
    input_table.SystemCodeNumber,
    input_table.LastUpdatedDate,
    input_table.LastUpdatedTime,
    occupancy_rate=pw.apply(lambda o, c: o / c if c else 0, input_table.Occupancy, input_table.Capacity),
    queue=input_table.QueueLength,
    traffic_num=pw.apply(lambda x: traffic_map.get(x, 2), input_table.TrafficConditionNearby),
    is_special=pw.apply(lambda x: 1 if x.lower() == "yes" else 0, input_table.IsSpecialDay),
    vehicle_weight=pw.apply(lambda x: vehicle_weights.get(x.lower(), 0), input_table.VehicleType)
)

with_demand = cleaned.select(
    cleaned.SystemCodeNumber,
    cleaned.LastUpdatedDate,
    cleaned.LastUpdatedTime,
    cleaned.occupancy_rate,
    cleaned.queue,
    cleaned.traffic_num,
    cleaned.is_special,
    cleaned.vehicle_weight,
    demand=pw.apply(
        lambda o, q, t, s, v: α * o + β * q - γ * t + δ * s + v,
        cleaned.occupancy_rate, cleaned.queue, cleaned.traffic_num, cleaned.is_special, cleaned.vehicle_weight
    )
)

final = with_demand.select(
    with_demand.SystemCodeNumber,
    with_demand.LastUpdatedDate,
    with_demand.LastUpdatedTime,
    with_demand.demand,
    norm_demand=pw.apply(lambda d: (d - 0.0) / (3.0 - 0.0 + 1e-6), with_demand.demand),
    price=pw.apply(lambda d: max(min(BASE_PRICE * (1 + λ * ((d - 0.0) / (3.0 - 0.0 + 1e-6))), 2 * BASE_PRICE), 0.5 * BASE_PRICE),
                  with_demand.demand)
)

In [34]:
pw.io.csv.write(final, filename="model2_stream_output.csv")
pw.run()

Output()

In [40]:
print(df.columns.tolist())

['t', 'lot', 'price', 'diff', 'time']


In [62]:
import pandas as pd

df_streamed = pd.read_csv("model2_stream_output.csv")

df_streamed["timestamp"] = pd.to_datetime(
    df_streamed["LastUpdatedDate"] + " " + df_streamed["LastUpdatedTime"],
    errors="coerce",
    dayfirst=True
)

df_streamed["date"] = df_streamed["timestamp"].dt.date
df_streamed["lot"] = df_streamed["SystemCodeNumber"]

daily_avg = df_streamed.groupby(["lot", "date"])["price"].mean().reset_index()
daily_avg["date_str"] = daily_avg["date"].astype(str)

In [67]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show, output_notebook
output_notebook()

plots = []

for lot_id in df_streamed["SystemCodeNumber"].unique():
    lot_df = df_streamed[df_streamed["SystemCodeNumber"] == lot_id].copy()
    lot_df = lot_df.sort_values("timestamp")
    source = ColumnDataSource(lot_df)

    p = figure(
        title=f"Model 2 Price Trend – Lot: {lot_id}",
        x_axis_type="datetime",
        width=900,
        height=300,
        tools="pan,box_zoom,reset,save"
    )

    p.line(x="timestamp", y="price", source=source, line_width=2, color="steelblue", legend_label="Price")
    p.circle(x="timestamp", y="price", source=source, size=4, color="steelblue")

    hover = HoverTool(
        tooltips=[
            ("Timestamp", "@timestamp{%F %H:%M}"),
            ("Price", "@price{0.00}")
        ],
        formatters={"@timestamp": "datetime"},
        mode="vline"
    )
    p.add_tools(hover)

    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price"
    p.legend.visible = False
    p.xaxis.major_label_orientation = 0.8

    plots.append(p)

# Display all plots in a scrollable column
show(column(*plots))